In [ ]:
# Description: This program uses an artificial recurrent neural network called Long Short Term Memory (LSTM)
#              to predict the closing stock price of a corporation (Apple Inc.) using the past 60 day stock price.

In [ ]:
# Install the libraries
!pip install numpy
!pip install pandas-datareader
!pip install yfinance
!pip install scikit-learn
!pip install keras
!pip install tensorflow
!pip install matplotlib
!pip install datetime

In [ ]:
# Import the libraries
import tensorflow.compat.v1 as tf                # A library which is used for machine learning models.
tf.disable_v2_behavior()                         # Disables TensorFlow version 2 behavior and enables version 1.
tf.compat.v1.disable_eager_execution()           # Disables the TensorFlow eager execution mode.
import numpy as np                               # A library for numerical computing in Python.
from pandas_datareader import data as pdr        # Alias for the pandas_datareader.data module.
import yfinance as yfin                          # A library for downloading financial data from Yahoo Finance.
from sklearn.preprocessing import MinMaxScaler   # A library for scaling numerical data.
from keras.models import Sequential              # A library for building neural network models.
from keras.layers import Dense, LSTM, Dropout    # Classes for defining layers in a neural network.
import matplotlib.pyplot as plt                  # A library for creating data visualizations in Python.

In [ ]:
# Get the data using the yfinance library and the pandas-datareader API
import datetime 
yfin.pdr_override()
df = pdr.get_data_yahoo('AAPL', start='2019-01-01', end=datetime.date.today())

# Show the data
df

In [ ]:
# Plot the closing price history
df['Close'].plot(figsize=(16,8), linewidth=2)
plt.title('Closing Price History')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.show

In [ ]:
# Create a new numpy array with only the 'Close' column
dataset = df[['Close']].values

# Calculate the number of rows to use for training
training_data_len = int(len(dataset) * 0.8)

training_data_len

In [ ]:
# Scale the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

In [ ]:
# Create the training dataset

# Create the scaled training dataset
train_data=scaled_data[0:training_data_len, :]

# Split the data into x_train and y_train datasets
x_train = [] # contain the input features for the model
y_train = [] # contain the target values

# Loop through the training data, creating x_train and y_train datasets
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])  # Append the past 60 values to the x_train list
    y_train.append(train_data[i, 0])       # Append the next value to the y_train list
    
# Convert the x_train and y_train to numpy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)

# Reshape the data to 3 dimensions because the LSTM model expects a 3D input shape
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
# Build the LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=True),
    Dropout(0.2),
    LSTM(50, return_sequences=True),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1)
])

# Compile the model with an optimizer and a loss function
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model with the training data and target values
model.fit(x_train, y_train, epochs=100, batch_size=64)

In [ ]:
# Create the testing dataset

# Create a new array containing scaled values
test_data = scaled_data[training_data_len - 60:, :]

# Create the datasets x_test and y_test
x_test = np.array([test_data[i-60:i, 0] for i in range(60, len(test_data))]) # will contain the input features for the model
y_test = dataset[training_data_len:, :] # will contain the actual stock prices for the testing data

# Convert the data to a numpy array to use it in the LSTM model
x_test = np.array(x_test)

# Reshape the data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
# Predict the stock prices using the trained LSTM model on the testing data
predictions = scaler.inverse_transform(model.predict(x_test))

In [ ]:
# Visualize the predicted values alongside the actual values

# Get the training and validation datasets
train = df[['Close']].iloc[:training_data_len]
valid = df[['Close']].iloc[training_data_len:]

# Add predicted values as a new column in the validation dataset
valid['Predictions'] = predictions

# Show the actual and predicted values
print(valid)

# Show the average error between the actual values and the predictions
avg_error = round(np.mean(np.abs(valid.iloc[:,0] - valid.iloc[:,1])), 2)
print(f"\nAverage error: {avg_error}%")

In [ ]:
# Plot the training and validation datasets, as well as the predicted values
plt.figure(figsize=(16,8))
plt.title('Model', fontsize=20)
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'], linewidth=2)
plt.plot(valid['Close'], linewidth=2)
plt.plot(valid['Predictions'], linewidth=2)
plt.legend(['Training values', 'Actual values', 'Predictions'], fontsize=14, loc='lower right')
plt.grid(True)
plt.show

In [ ]:
# Select the last 60 closing prices from the 'Close' column and reshape the array to have a single feature
last_60_days = df['Close'][-60:].values.reshape(-1,1)

# Scale the data using the same scaler used on the training and testing data
last_60_days_scaled = scaler.transform(last_60_days)

# Predict the scaled price for the next day using the LSTM model
predicted_price_scaled_tomorrow = model.predict(np.array([last_60_days_scaled]))

# Reverse the scaling to get the predicted price in dollars.
predicted_price_tomorrow = scaler.inverse_transform(predicted_price_scaled_tomorrow)

# Print the predicted price for the next day and next week
tomorrow = datetime.date.today() + datetime.timedelta(days=1)
print(f"The predicted closing price for tomorrow ({tomorrow}) is: {round(float(predicted_price_tomorrow), 2)} $")